In [3]:
%%writefile PIT.py
import math
import meep as mp
from meep import mpb
#!rm *.h5
num_bands = 3

#k_points = [mp.Vector3(),
#            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
#            mp.Vector3(0.0  ,0.625,0.375),  #U
#            mp.Vector3(0.0  ,0.5  ,0.0  ),  #L
#            mp.Vector3(0.0  ,0.0  ,0.0  ),  #Gamma
#            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
#            mp.Vector3(0.25 ,0.75 ,0.5  ),  #W
#            mp.Vector3(0.375,0.75 ,0.375),  #K
#            mp.Vector3()]          

k_points = [mp.Vector3(),
            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
            mp.Vector3(0.5  ,0.625,0.625),  #U
            mp.Vector3(0.5  ,0.5  ,0.5  ),  #L
            mp.Vector3(0.0  ,0.0  ,0.0  ),  #Gamma
            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
            mp.Vector3(0.25 ,0.75 ,0.5  ),  #W
            mp.Vector3(0.375,0.75 ,0.375),  #K
            mp.Vector3()]  

res0 = 16
n0 = 1.5
tol = 0.001
mesh_size = 3
L = math.sqrt(3)/4
n = math.sqrt(32)
r = 0.0725
m = mp.Medium(epsilon=n*n)

geometry = [mp.Cylinder(center=mp.Vector3(0,0,0),radius=r, material=m, height=L, axis=mp.Vector3(1,1,1)),
           mp.Cylinder(center=mp.Vector3(0.5,0,0),radius=r, material=m, height=L, axis=mp.Vector3(3,-1,-1)),
           mp.Cylinder(center=mp.Vector3(0,0.5,0),radius=r, material=m, height=L, axis=mp.Vector3(-1,3,-1)),
           mp.Cylinder(center=mp.Vector3(0,0,0.5),radius=r, material=m, height=L, axis=mp.Vector3(-1,-1,3))]

#a = math.sqrt(0.5)
#geometry_lattice = mp.Lattice(basis-size=mp.Vector3(a,a,a),
#                                 basis1=mp.Vector3(0,1,1),
#                                 basis2=mp.Vector3(1,0,1),
#                                 basis3=mp.Vector3(1,1,0))

sqrt_half = math.sqrt(0.5)
geometry_lattice = mp.Lattice(
    basis_size=mp.Vector3(sqrt_half, sqrt_half, sqrt_half),
    basis1=mp.Vector3(0, 1, 1),
    basis2=mp.Vector3(1, 0, 1),
    basis3=mp.Vector3(1, 1)
)
k_points = mp.interpolate(4, k_points)

ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=res0*n/n0,
                    mesh_size = mesh_size)
#mpb.ModeSolver.optimize_grid_size()
ms.run()
ms.output_epsilon()

Overwriting PIT.py


In [ ]:
ms.band_range_data

In [4]:
!rm PI.dat
!python PIT.py >& PI.out
!grep freqs PI.out >> PI.dat

In [ ]:
!rm *.h5
!rm *.vtk
!python PIT.py 
!h5tovtk PIT-epsilon.h5

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
num_bands=3
nip_raw = []
k_raw = []
omega_raw = []

with open('PI.dat') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_file)
    for row in csv_reader:
        nip_raw.append(row[1])
        k_raw.append(row[2:5])
        omega_raw.append(row[6:num_bands+6])
        

In [ ]:
N=np.size(nip_raw)
nip=np.zeros((N))
k=np.zeros((N,3))
omega=np.zeros((N,num_bands))
for i in range(0,N):
    nip[i] = float(nip_raw[i])
    for j in range(0,3):
        k[i][j] = float(k_raw[i][j])
        
    for j in range(0,num_bands):
        omega[i][j] = float(omega_raw[i][j])

plt.plot(nip,omega[:,:])

In [ ]:
import math
import meep as mp
from meep import mpb
num_bands = 8
k_points = [mp.Vector3(),          # Gamma
            mp.Vector3(0.5),       # X
            mp.Vector3(0.5, 0.5),  # M
            mp.Vector3()]          # Gamma
k_points = mp.interpolate(4, k_points)
geometry = [mp.Cylinder(0.2, material=mp.Medium(epsilon=12))]
geometry_lattice = mp.Lattice(size=mp.Vector3(1, 1))
resolution = 32
ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=resolution)
print("Square lattice of rods: TE bands")
ms.run_tm()
ms.output_epsilon()

In [ ]:
#!python PIT.py >& PI.out
#!rm PI.dat
#!rm *.h5
#!grep tmfreqs PI.out >> PI.dat

import csv
import numpy as np
import matplotlib.pyplot as plt
num_bands=8
nip_raw = []
k_raw = []
omega_raw = []

with open('PI.dat') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_file)
    for row in csv_reader:
        nip_raw.append(row[1])
        k_raw.append(row[2:5])
        omega_raw.append(row[6:num_bands+6])
        
N=np.size(nip_raw)
nip=np.zeros((N))
k=np.zeros((N,3))
omega=np.zeros((N,num_bands))
for i in range(0,N):
    nip[i] = float(nip_raw[i])
    for j in range(0,3):
        k[i][j] = float(k_raw[i][j])
        
    for j in range(0,num_bands):
        omega[i][j] = float(omega_raw[i][j])

plt.plot(nip,omega[:,:])

In [ ]:
import math
(0.57735/0.408)*math.sqrt(2)

In [ ]:
!python PIT.py 
!h5tovtk PIT-epsilon.h5